# CHAPTER 5 - The Simple Recurrent Neural Network

_pg. 67-82_

# PART 1
_pg.70-76_

## Prepare the Sample Data

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

In [3]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

loc = "/home/ubuntu/CSU/Notebooks/csu-2017/COE.csv"
temp = pd.read_csv(loc)
data = temp.drop(temp.columns [[0,1]], axis=1)

y = data["COE$"]

x = data.drop(data.columns[[0,4]], axis=1)
x = x.apply(np.log)
x = pd.concat([x, data["Open?"]], axis=1)

scaler_x = preprocessing.MinMaxScaler(feature_range=(0,1))
x = np.array(x).reshape((len(x), 4))
x = scaler_x.fit_transform(x)

scaler_y = preprocessing.MinMaxScaler(feature_range=(0,1))
y = np.array(y).reshape((len(y), 1))
y = np.log(y)
y = scaler_y.fit_transform(y)

## Follow this Approach to Create Your Train and Test Sets

In [4]:
end = len(x) - 1
learn_end = int(end * 0.954)
x_train = x[0:learn_end - 1, ]
x_test = x[learn_end:end - 1, ]
y_train = y[1:learn_end]
y_test = y[learn_end + 1: end]

x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))

In [5]:
print "Shape of x_train is ", x_train.shape

Shape of x_train is  (250, 4, 1)


In [6]:
print "Shape of x_test is ", x_test.shape

Shape of x_test is  (12, 4, 1)


## Import Keras

In [7]:
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import SimpleRNN

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: unknown error)


## Determine Model Structure

In [9]:
seed = 2016
np.random.seed(seed)
fit1 = Sequential()
# UPDATED
fit1.add(SimpleRNN(output_dim=8, activation="tanh", input_shape=(4,1)))
# fit1.add(SimpleRNN(units=8, activation="tanh", input_shape=(4,1)))
# UPDATED
fit1.add(Dense(output_dim=1, activation='linear'))
fit1.add(Dense(units=1, activation='linear'))

TypeError: __init__() takes at least 2 arguments (2 given)

# PART 2
_pg. 79-81_

In [ ]:
# uses accelerated gradient descent
sgd = SGD(lr=0.0001, momentum=0.95, nesterov=True)
fit1.compile(loss='mean_squared_error', optimizer=sgd)

## Fitting the Model
The model, with a batch size of 10, is fit over 700 epochs as follows:

In [ ]:
%%capture
fit1.fit(x_train, y_train, batch_size=10, epochs=700)

In [ ]:
score_train = fit1.evaluate(x_train, y_train, batch_size=10)
score_test = fit1.evaluate(x_test, y_test, batch_size=10)
print "in train MSE = ", round(score_train, 6)
print "in test MSE = ", round(score_test, 6)

In [ ]:
pred1 = fit1.predict(x_test)
print pred1

In [ ]:
# apply the inverse transformation to predictions
pred1 = scaler_y.inverse_transform(np.array(pred1).reshape((len(pred1), 1)))
print pred1

In [ ]:
# undo the initial log transformation of the data
pred1 = np.exp(pred1)
print np.rint(pred1)

## Visualize the Data

In [ ]:
# CREATE FIGURE 5.4
# pg. 81
plt.rcParams['figure.figsize'] = (8,6)

# Original Series
obs_price = data["COE$"][-13:-1]
max_price = (obs_price + 1500).tolist()
min_price = (obs_price - 1500).tolist()
obs_price.tolist()

plt.plot(range(12), obs_price, linestyle='solid', label="Observed", color="darkblue", linewidth='1')

# Predictions
plt.plot(range(12), pred1, linestyle='solid', label="Predicted", color="red", linewidth='1')

# Desired Tolerance
plt.plot(range(12), max_price, linestyle='solid', label="Tolerance", color="grey", linewidth='0.5')
plt.plot(range(12), min_price, linestyle='solid', label=None, color="grey", linewidth='0.5')

# Figure Settings
plt.title("Figure 5.4: Actual and forecasted values for Simple RNN")
plt.xlabel("Prediction Period")
plt.ylabel("Singaporean dollars")
plt.ylim(9000,18000)
plt.legend()
plt.show()